In [145]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [146]:
def convert_columns_to_str(df,number_columns):
    """
    Функция для конвертации указанных столбцов в строковый тип и очистки от пробельных символов в начале и конце
    """
    
    for column in number_columns: # Перебираем список нужных колонок
        df.iloc[:,column] = df.iloc[:,column].astype(str)
        # Очищаем колонку от пробельных символов с начала и конца
        df.iloc[:,column] = df.iloc[:,column].apply(lambda x: x.strip())



In [147]:
file_params = 'params.xlsx'
path_to_end_folder = 'data/'

In [148]:
first_df = pd.read_excel('data/Большой список.xlsx')
second_df = pd.read_excel('data/Малый список.xlsx')
# Считываем файл параметров без заголовка, чем проще тем лучше
params = pd.read_excel(file_params,header=None)

In [149]:
params

,0,1
0,1,9
1,2,2
2,3,3
3,4,4


In [150]:
# Преврашаем каждую колонку в список
params_first_columns = params[0].tolist()
params_second_columns = params[1].tolist()



In [151]:
# Конвертируем в инт заодно проверяя корректность введенных данных
try:
    int_params_first_columns = list(map(int,params_first_columns))
    int_params_second_columns = list(map(int,params_second_columns))
except:
    print('В колонках файла параметров должны быть только цифры!!!')
    


In [152]:
# Отнимаем 1 от каждого значения чтобы привести к питоновским индексам
int_params_first_columns = list(map(lambda x:x-1,int_params_first_columns))
int_params_second_columns = list(map(lambda x:x-1,int_params_second_columns))

In [153]:
# Конвертируем нужные нам колонки в str
convert_columns_to_str(first_df,int_params_first_columns)
convert_columns_to_str(second_df,int_params_second_columns)

In [154]:
# Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
first_df['ID'] = first_df.iloc[:,int_params_first_columns].sum(axis=1)
second_df['ID'] = second_df.iloc[:,int_params_second_columns].sum(axis=1)


In [155]:
first_df['ID']

0            1АлехинДанилаПрокопьевич
1            1АлехинДанилаПрокопьевич
2           3ЭсауловаВалерияПрокловна
3               4ПанинГавриилВласович
4       5ГрибановКонстантинСтепанович
                    ...              
998     999НиконоваАнтонинаМаксимовна
999     1000ФурмановАркадийЕлизарович
1000     1001КорниецЛюдмилаМаксимовна
1001    1002СеребровГавриилФеодосивич
1002         1003ЦойВалентинЗахарович
Name: ID, Length: 1003, dtype: object

In [156]:
second_df['ID']

0           1АлехинДанилаПрокопьевич
1          2ЭсауловаВалерияПрокловна
2              3ПанинГавриилВласович
3      4ГрибановКонстантинСтепанович
4          5МандрыкоТрофимЛеонидович
                   ...              
436    437НиконоваАнтонинаМаксимовна
437     438ФурмановАркадийЕлизарович
438      439КорниецЛюдмилаМаксимовна
439     440СеребровГавриилФеодосивич
440     441СеребровГавриилФеодосивич
Name: ID, Length: 441, dtype: object

In [157]:
# Обрабатываем дубликаты

duplicates_first_df = first_df[first_df.duplicated(subset=['ID'],keep=False)] # Сохраняем все значения у которых есть дубликаты в отдельный датафрейм


first_df.drop_duplicates(subset=['ID'],keep=False,inplace=True)# Удаляем дубликаты из датафрейма

duplicates_second_df = second_df[second_df.duplicated(subset=['ID'],keep=False)] # Сохраняем все значения у которых есть дубликаты в отдельный датафрейм
second_df.drop_duplicates(subset=['ID'],keep=False,inplace=True)# Удаляем дубликаты из датафрейма


In [158]:
duplicates_first_df

,1,Фамилия,Имя,Отчество,ФИО,Дата рождения,Номер телефона,Электронная почта,Адрес прописки,Страна,...,ИНН ФЗ/ИП,СНИЛС,ОМС,Специальность,Направление,Учебное заведение,Серия/Номер диплома,Регистрационный номер,Дата окончания обучения,ID
0,1,Алехин,Данила,Прокопьевич,Алехин Данила Прокопьевич,14.04.1973,NaN,NaN,"Россия, г. Иркутск, Кирова ул., д. 1 кв.212",Россия,...,158493217864,NaN,5950614844448865,Администратор базы данных,02.03.03 Математическое обеспечение и админист...,Санкт-Петербургский государственный университет,714531 1469061,11-701-701,2019,1АлехинДанилаПрокопьевич
1,1,Алехин,Данила,Прокопьевич,Алехин Данила Прокопьевич,14.04.1973,NaN,NaN,"Россия, г. Иркутск, Кирова ул., д. 1 кв.212",Россия,...,158493217864,NaN,5950614844448865,Администратор базы данных,02.03.03 Математическое обеспечение и админист...,Санкт-Петербургский государственный университет,714531 1469061,11-701-701,2019,1АлехинДанилаПрокопьевич


In [159]:
# Создаем документ
wb = openpyxl.Workbook()
# создаем листы
ren_sheet = wb['Sheet']
ren_sheet.title = 'Данные которых нет во 2 таблице'
wb.create_sheet(title='Данные которых нет во 1 таблице', index=1)
wb.create_sheet(title='Совпадающие данные', index=2)
# Создаем листы для дубликатов
wb.create_sheet(title='Дубликаты первая таблица', index=3)
wb.create_sheet(title='Дубликаты вторая таблица', index=4)

# Создаем датафрейм
itog_df = pd.merge(first_df, second_df, how='outer', left_on=['ID'], right_on=['ID'],
                   indicator=True)

# Записываем каждый датафрейм в соответсвующий лист
left_df = itog_df[itog_df['_merge'] == 'left_only']


left_df.drop(['_merge'], axis=1, inplace=True)
for r in dataframe_to_rows(left_df, index=False, header=True):
    wb['Данные которых нет во 2 таблице'].append(r)

right_df = itog_df[itog_df['_merge'] == 'right_only']
right_df.drop(['_merge'], axis=1, inplace=True)
for r in dataframe_to_rows(right_df, index=False, header=True):
    wb['Данные которых нет во 1 таблице'].append(r)

both_df = itog_df[itog_df['_merge'] == 'both']
both_df.drop(['_merge'], axis=1, inplace=True)
for r in dataframe_to_rows(both_df, index=False, header=True):
    wb['Совпадающие данные'].append(r)

# Записываем дубликаты в соответствующие листы
for r in dataframe_to_rows(duplicates_first_df, index=False, header=True):
    wb['Дубликаты первая таблица'].append(r)

for r in dataframe_to_rows(duplicates_second_df, index=False, header=True):
    wb['Дубликаты вторая таблица'].append(r)

In [160]:
# Сохраняем
t = time.localtime()
current_time = time.strftime('%H_%M_%S %d.%m.%Y', t)
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder}/Совпадающие,уникальные данные  от {current_time}.xlsx')